<a href="https://colab.research.google.com/github/Dark-Sied/Intent_Classification/blob/master/Intent_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


In [4]:
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "latin1", names = ["Sentence", "Intent"])
  print(df.head())
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)
  


In [5]:
intent, unique_intent, sentences = load_dataset("Dataset.csv")

                Sentence          Intent
0       Need help pleese  commonQ.assist
1              Need help  commonQ.assist
2       I need some info  commonQ.assist
3      Will you help me?  commonQ.assist
4  What else can you do?  commonQ.assist


In [6]:
print(sentences[:5])

['Need help pleese', 'Need help', 'I need some info', 'Will you help me?', 'What else can you do?']


In [7]:
# nltk.download("stopwords")
# nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/karthik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/karthik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
#define stemmer
stemmer = LancasterStemmer()

In [9]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    #stemming
    words.append([i.lower() for i in w])
    
  return words  

In [10]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
  


1113
[['need', 'help', 'pleese'], ['need', 'help']]


In [38]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  print(token)
  return token

In [12]:
def max_length(words):
  return(len(max(words, key = len)))
  

In [ ]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
print(word_tokenizer.word_index)
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

In [ ]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [41]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)
print(encoded_doc)

[[25, 77, 332], [25, 77], [1, 25, 198, 181], [51, 10, 77, 16], [8, 268, 4, 10, 30], [8, 30, 10, 30], [8, 4, 10, 77, 16, 60], [8, 4, 10, 30], [8, 39, 10, 182, 134], [25, 77, 333], [11, 4, 10, 77, 16], [4, 10, 77, 16], [11, 4, 10, 269], [11, 4, 10, 77, 16], [11, 30, 10, 77], [11, 6, 77, 10], [11, 51, 10, 269, 16], [1, 25, 77], [4, 10, 27, 198, 181], [77, 16, 35], [7, 23, 21, 143], [7, 89, 21, 143, 199, 6, 16], [7, 83, 270, 271], [228, 272, 334], [143, 144, 145], [143, 7, 199, 60, 16], [39, 10, 229, 273, 271], [39, 10, 274], [39, 10, 200], [39, 10, 183, 228], [39, 10, 21, 274], [39, 10, 21, 160], [39, 10, 21, 200], [39, 10, 21, 275], [39, 10, 21, 143], [39, 335, 270, 336], [161, 1, 337, 6, 21, 143], [10, 39, 21, 338, 39, 339], [10, 39, 21, 143, 144, 145], [23, 7, 21, 200], [23, 7, 21, 275], [229, 273, 340], [230, 10, 229, 341], [7, 23, 228, 272, 184], [7, 23, 21, 200], [39, 10, 21, 145], [342, 343], [11, 344, 31, 345], [11, 10, 346], [276, 109], [8, 78, 347], [348, 349, 350], [351], [11, 

In [16]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [17]:
padded_doc = padding_doc(encoded_doc, max_length)

In [18]:
padded_doc[:5]

array([[ 25,  77, 332,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 25,  77,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1,  25, 198, 181,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 51,  10,  77,  16,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  8, 268,   4,  10,  30,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int32)

In [19]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (1113, 28)


In [20]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


In [21]:
output_tokenizer.word_index

{'commonq.not_giving': 1,
 'faq.bad_service': 2,
 'commonq.how': 3,
 'faq.aadhaar_missing': 4,
 'faq.application_process': 5,
 'faq.biz_new': 6,
 'commonq.just_details': 7,
 'faq.apply_register': 8,
 'faq.biz_category_missing': 9,
 'faq.borrow_limit': 10,
 'faq.biz_simpler': 11,
 'commonq.wait': 12,
 'commonq.query': 13,
 'faq.approval_time': 14,
 'commonq.assist': 15,
 'commonq.bot': 16,
 'contact.contact': 17,
 'faq.borrow_use': 18,
 'faq.banking_option_missing': 19,
 'commonq.name': 20,
 'faq.address_proof': 21}

In [43]:
encoded_output = encoding_doc(output_tokenizer, intent)
print(encoded_output)

[[15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [15], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [16], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [7], [7], [7], [7], [7], [7], [20], [20], [20], [20], [20], [20], [20], [20], [20], [20], [20], [20], [20], [20], [20], [20], [20], [20], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [13], [13], [13], [13], [13], [13], [13], [12], [12], [12], [12], [12], [12], [12], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [17], [1

In [23]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [24]:
encoded_output.shape

(1113, 1)

In [25]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [26]:
output_one_hot = one_hot(encoded_output)

/home/karthik/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [27]:
output_one_hot.shape

(1113, 21)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)


In [30]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (890, 28) and train_Y = (890, 21)
Shape of val_X = (223, 28) and val_Y = (223, 21)


In [31]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(21, activation = "softmax"))
  
  return model

In [32]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           62976     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                693       
Total params: 335,061
Trainable params: 272,085
Non-trainable params: 62,976
_________________________________________________________________


In [33]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Train on 890 samples, validate on 223 samples
Epoch 1/100
890/890 [==============================] - 5s 5ms/step - loss: 3.0032 - acc: 0.0978 - val_loss: 2.8962 - val_acc: 0.1659

Epoch 00001: val_loss improved from inf to 2.89621, saving model to model.h5
Epoch 2/100
890/890 [==============================] - 2s 2ms/step - loss: 2.8974 - acc: 0.1427 - val_loss: 2.7963 - val_acc: 0.2242

Epoch 00002: val_loss improved from 2.89621 to 2.79633, saving model to model.h5
Epoch 3/100
890/890 [==============================] - 2s 2ms/step - loss: 2.8350 - acc: 0.1562 - val_loss: 2.7177 - val_acc: 0.1659

Epoch 00003: val_loss improved from 2.79633 to 2.71765, saving model to model.h5
Epoch 4/100
890/890 [==============================] - 2s 2ms/step - loss: 2.7869 - acc: 0.1573 - val_loss: 2.5803 - val_acc: 0.1749

Epoch 00004: val_loss improved from 2.71765 to 2.58032, saving model to model.h5
Epoch 5/100
890/890 [==============================] - 2s 2ms/step - loss: 2.6619 - acc: 0.1831 - 

890/890 [==============================] - 2s 2ms/step - loss: 1.0222 - acc: 0.6843 - val_loss: 0.9790 - val_acc: 0.6951

Epoch 00040: val_loss did not improve from 0.92801
Epoch 41/100
890/890 [==============================] - 2s 2ms/step - loss: 0.9275 - acc: 0.7067 - val_loss: 0.9824 - val_acc: 0.7444

Epoch 00041: val_loss did not improve from 0.92801
Epoch 42/100
890/890 [==============================] - 2s 2ms/step - loss: 0.8691 - acc: 0.7315 - val_loss: 0.9973 - val_acc: 0.7534

Epoch 00042: val_loss did not improve from 0.92801
Epoch 43/100
890/890 [==============================] - 2s 2ms/step - loss: 0.8426 - acc: 0.7438 - val_loss: 0.9708 - val_acc: 0.7444

Epoch 00043: val_loss did not improve from 0.92801
Epoch 44/100
890/890 [==============================] - 2s 2ms/step - loss: 0.8439 - acc: 0.7303 - val_loss: 0.9071 - val_acc: 0.7399

Epoch 00044: val_loss improved from 0.92801 to 0.90713, saving model to model.h5
Epoch 45/100
890/890 [==============================]

890/890 [==============================] - 2s 2ms/step - loss: 0.4850 - acc: 0.8438 - val_loss: 0.8323 - val_acc: 0.7892

Epoch 00083: val_loss did not improve from 0.74497
Epoch 84/100
890/890 [==============================] - 2s 2ms/step - loss: 0.4608 - acc: 0.8506 - val_loss: 0.8603 - val_acc: 0.7982

Epoch 00084: val_loss did not improve from 0.74497
Epoch 85/100
890/890 [==============================] - 2s 2ms/step - loss: 0.4633 - acc: 0.8461 - val_loss: 0.8224 - val_acc: 0.7937

Epoch 00085: val_loss did not improve from 0.74497
Epoch 86/100
890/890 [==============================] - 2s 2ms/step - loss: 0.4744 - acc: 0.8371 - val_loss: 0.8365 - val_acc: 0.8072

Epoch 00086: val_loss did not improve from 0.74497
Epoch 87/100
890/890 [==============================] - 2s 2ms/step - loss: 0.4663 - acc: 0.8404 - val_loss: 0.8672 - val_acc: 0.7803

Epoch 00087: val_loss did not improve from 0.74497
Epoch 88/100
890/890 [==============================] - 2s 2ms/step - loss: 0.4606 

In [34]:
 model = load_model("model.h5")

In [35]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred


  

In [36]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [37]:
text = "can you get sum of all marks"
pred = predictions(text)
get_final_output(pred, unique_intent)

['can', 'you', 'get', 'sum', 'of', 'all', 'marks']
contact.contact has confidence = 0.37510538
faq.biz_new has confidence = 0.12876078
commonQ.bot has confidence = 0.09474806
faq.apply_register has confidence = 0.08270825
commonQ.assist has confidence = 0.066969074
commonQ.name has confidence = 0.0667417
commonQ.query has confidence = 0.038959064
faq.application_process has confidence = 0.032878432
commonQ.how has confidence = 0.027631992
commonQ.wait has confidence = 0.025153242
faq.bad_service has confidence = 0.013217069
faq.address_proof has confidence = 0.009759407
commonQ.not_giving has confidence = 0.008911743
faq.biz_simpler has confidence = 0.00804339
commonQ.just_details has confidence = 0.007985479
faq.aadhaar_missing has confidence = 0.0046423054
faq.borrow_limit has confidence = 0.0028019561
faq.banking_option_missing has confidence = 0.0024704395
faq.borrow_use has confidence = 0.0015535804
faq.biz_category_missing has confidence = 0.0006133006
faq.approval_time has confi